Here we will connect 2 types of dfs:
1. The matches db that includes the __names__ of the players
2. The player ratings db that inclueds also the __names__ of the players

We will connect via the closest string for each player in a match in order to get:
1. player rating
2. player age
3. player height

In [34]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

First we need to fix the club name issue, for exmaple Man utd in the matches db is not the same as Manchester United

In [35]:
def extract_text_from_html(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')
    return soup.get_text(strip=True)

def fix_html_col(df, col):
    df[col] = df[col].apply(extract_text_from_html)


We will see the names of the clubs in the season 22-23

And now we see which clubs dont have the names in the players db

In [36]:
def show_different_names(df1, df2):
    unique_home_team_names = set(df1['home_team_name'].unique())
    club_names = set(df2['Club Name'].unique())
    diff_names = unique_home_team_names.difference(club_names)
    return diff_names


processing: All we need is the full name, club, age, overall and height

Secondly we will use the player name and club name as a key, to replace player names in the matches DB with numbers we can work with

to be continued: after scraping seasons

In [37]:
def find_most_similar_name(target_name, names_list, threshold=0.25):
    similarities = [(other_name, SequenceMatcher(None, target_name, other_name).ratio()) for other_name in names_list]
    most_similar_name, similarity_score = max(similarities, key=lambda x: x[1])
    
    if similarity_score >= threshold:
        #print('found player '+most_similar_name+" with a score of "+str(similarity_score))
        return most_similar_name
    else:
        #print("didn't find", target_name)
        return None

In [38]:
def find_player(player_name, club_name,  df, attributes):
    #first we filter by club name
    temp = df[df['Club Name'] == club_name]

    #Now we find the player
    sim_name = find_most_similar_name(player_name, temp['Full Name'])
    
    return temp[temp['Full Name'] == sim_name].iloc[0][attributes] if sim_name else None


What will be our key?

Since using just the name will lead to issues (there are players with the same name) then we should also use the name AND the club as the key.

How can we do it?



In [39]:
def ratings_col(df, att_df):
    '''
    puts for every player their 'Overall', 'Age', 'Height(in cm)', 'Weight(in kg)'
    for each player i we will have the column
    HomePlayeri (if the player is on the home team)
    AwayPlayeri (if the player is on the away team)
    and we will use the home_team_name or away_team_name and the full name as the key (we will use find_player(player_name, club_name,  df, attributes))
    '''

    attributes = [ 'Overall', 'Age', 'Height(in cm)', 'Weight(in kg)']
    # Iterate through each player column
    for i in range(1, 21):

        home_col = f'HomePlayer{i}'
        away_col = f'AwayPlayer{i}'
        if home_col in df.columns or away_col in df.columns:
            # Add columns for home team players
            for att in attributes:
                df[home_col + "_" + att] = df.apply(
                    lambda row: 0 if pd.isna(row[home_col]) else
                    (find_player(row[home_col], row['home_team_name'], att_df, attributes)[att]
                    if find_player(row[home_col], row['home_team_name'], att_df, attributes) is not None else None),
                    axis=1
                )
            # df[home_col + '_Pace'] = df.apply(lambda row: 0 if pd.isna(row[home_col]) else find_player(row[home_col], row['home_team_name'], att_df, attributes)['Pace Total'] if find_player(row[home_col], row['home_team_name'], att_df, attributes) is not None else None, axis=1)
            # df[home_col + '_Def'] = df.apply(lambda row: 0 if pd.isna(row[home_col]) else find_player(row[home_col], row['home_team_name'], att_df, attributes)['Defending Total'] if find_player(row[home_col], row['home_team_name'], att_df, attributes) is not None else None, axis=1)
            # df[home_col + '_Pass'] = df.apply(lambda row: 0 if pd.isna(row[home_col]) else find_player(row[home_col], row['home_team_name'], att_df, attributes)['Passing Total'] if find_player(row[home_col], row['home_team_name'], att_df, attributes) is not None else None, axis=1)
            # df[home_col + '_Physic'] = df.apply(lambda row: 0 if pd.isna(row[home_col]) else find_player(row[home_col], row['home_team_name'], att_df, attributes)['Physicality Total'] if find_player(row[home_col], row['home_team_name'], att_df, attributes) is not None else None, axis=1)
            # df[home_col + '_Shooting'] = df.apply(lambda row: 0 if pd.isna(row[home_col]) else find_player(row[home_col], row['home_team_name'], att_df, attributes)['Shooting Total'] if find_player(row[home_col], row['home_team_name'], att_df, attributes) is not None else None, axis=1)
            # df[home_col + '_Dribbling'] = df.apply(lambda row: 0 if pd.isna(row[home_col]) else find_player(row[home_col], row['home_team_name'], att_df, attributes)['Dribbling Total'] if find_player(row[home_col], row['home_team_name'], att_df, attributes) is not None else None, axis=1)
            # df[home_col + '_Overall'] = df.apply(lambda row: 0 if pd.isna(row[home_col]) else find_player(row[home_col], row['home_team_name'], att_df, attributes)['Overall'] if find_player(row[home_col], row['home_team_name'], att_df, attributes) is not None else None, axis=1)
            # df[home_col + '_Age'] = df.apply(lambda row: 0 if pd.isna(row[home_col]) else find_player(row[home_col], row['home_team_name'], att_df, attributes)['Age'] if find_player(row[home_col], row['home_team_name'], att_df, attributes) is not None else None, axis=1)
            # df[home_col + '_Height'] = df.apply(lambda row: 0 if pd.isna(row[home_col]) else find_player(row[home_col], row['home_team_name'], att_df, attributes)['Height(in cm)'] if find_player(row[home_col], row['home_team_name'], att_df, attributes) is not None else None, axis=1)
            # df[home_col + '_Weight'] = df.apply(lambda row: 0 if pd.isna(row[home_col]) else find_player(row[home_col], row['home_team_name'], att_df, attributes)['Weight(in kg)'] if find_player(row[home_col], row['home_team_name'], att_df, attributes) is not None else None, axis=1)

            # Add columns for away team players
            for att in attributes:
                df[away_col + "_" + att] = df.apply(
                    lambda row: 0 if pd.isna(row[away_col]) else
                    (find_player(row[away_col], row['away_team_name'], att_df, attributes)[att]
                    if find_player(row[away_col], row['away_team_name'], att_df, attributes) is not None else None),
                    axis=1
                )
            # df[away_col + '_Pace'] = df.apply(lambda row: 0 if pd.isna(row[home_col]) else find_player(row[home_col], row['away_team_name'], att_df, attributes)['Pace Total'] if find_player(row[away_col], row['away_team_name'], att_df, attributes) is not None else None, axis=1)
            # df[away_col + '_Def'] = df.apply(lambda row: 0 if pd.isna(row[home_col]) else find_player(row[home_col], row['away_team_name'], att_df, attributes)['Defending Total'] if find_player(row[away_col], row['away_team_name'], att_df, attributes) is not None else None, axis=1)
            # df[away_col + '_Pass'] = df.apply(lambda row: 0 if pd.isna(row[home_col]) else find_player(row[home_col], row['away_team_name'], att_df, attributes)['Passing Total'] if find_player(row[away_col], row['away_team_name'], att_df, attributes) is not None else None, axis=1)
            # df[away_col + '_Physic'] = df.apply(lambda row: 0 if pd.isna(row[home_col]) else find_player(row[home_col], row['away_team_name'], att_df, attributes)['Physicality Total'] if find_player(row[away_col], row['away_team_name'], att_df, attributes) is not None else None, axis=1)
            # df[away_col + '_Shooting'] = df.apply(lambda row: 0 if pd.isna(row[home_col]) else find_player(row[home_col], row['away_team_name'], att_df, attributes)['Shooting Total'] if find_player(row[away_col], row['away_team_name'], att_df, attributes) is not None else None, axis=1)
            # df[away_col + '_Dribbling'] = df.apply(lambda row: 0 if pd.isna(row[home_col]) else find_player(row[home_col], row['away_team_name'], att_df, attributes)['Dribbling Total'] if find_player(row[away_col], row['away_team_name'], att_df, attributes) is not None else None, axis=1)
            # df[away_col + '_Overall'] = df.apply(lambda row: 0 if pd.isna(row[away_col]) else find_player(row[away_col], row['away_team_name'], att_df, attributes)['Overall'] if find_player(row[away_col], row['away_team_name'], att_df, attributes) is not None else None, axis=1)
            # df[away_col + '_Age'] = df.apply(lambda row: 0 if pd.isna(row[away_col]) else find_player(row[away_col], row['away_team_name'], att_df, attributes)['Age'] if find_player(row[away_col], row['away_team_name'], att_df, attributes) is not None else None, axis=1)
            # df[away_col + '_Height'] = df.apply(lambda row: 0 if pd.isna(row[away_col]) else find_player(row[away_col], row['away_team_name'], att_df, attributes)['Height(in cm)'] if find_player(row[away_col], row['away_team_name'], att_df, attributes) is not None else None, axis=1)
            # df[away_col + '_Weight'] = df.apply(lambda row: 0 if pd.isna(row[away_col]) else find_player(row[away_col], row['away_team_name'], att_df, attributes)['Weight(in kg)'] if find_player(row[away_col], row['away_team_name'], att_df, attributes) is not None else None, axis=1)


In [40]:
def replace_nas(df):
    '''
    gets df, goes to the Age column and replaces None with 18, and replaces Weight(in kg) and Height(in cm) with the mean, and overall with the min value
    '''

    # 'Dribbling Total', 'Pace Total', 'Defending Total', 'Shooting Total', 'Physicality Total', 'Passing Total' our new fatures
    attributes = ['Overall','Height(in cm)', 'Weight(in kg)']
    # First, deal with the None values
    for i in range(1,21):
        home_col = f'HomePlayer{i}'
        away_col = f'AwayPlayer{i}'
        if home_col in df.columns:
            # Age assumption
            df[home_col+"_Age"].fillna(18, inplace=True)

            for att  in attributes: 
                non_zero_col = df[home_col+'_'+att].replace(0, None)
                df[home_col+'_'+att].fillna(non_zero_col.min())
        if away_col in df.columns:
            # Age assumption
            df[away_col+"_Age"].fillna(18, inplace=True)

            for att  in attributes: 
                non_zero_col = df[away_col+'_'+att].replace(0, None)
                df[away_col+'_'+att].fillna(non_zero_col.min())
            # df[home_col+"_Age"].fillna(18, inplace=True)
            
            # # dealing with some ratings

            # df[home_col+"_Total"].fillna(18, inplace=True)
            # df[home_col+"_Dribbling Total"].fillna(df[home_col+"_Dribbling Total"].replace(0,None).min(), inplace=True)
            # df[home_col+"_Defending Total"].fillna(df[home_col+"_Defending Total"].replace(0,None).min(), inplace=True)
            # df[home_col+"_Shooting Total"].fillna(df[home_col+"_Shooting Total"].replace(0,None).min(), inplace=True)
            # df[home_col+"_Physicality Total"].fillna(df[home_col+"_Physicality Total"].replace(0,None).min(), inplace=True)
            # df[home_col+"_Pace Total"].fillna(df[home_col+"_Pace Total"].replace(0,None).min(), inplace=True)

            # # Build assumption (ignoring zeros)
            # non_zero_weights = df[home_col+"_Weight(in kg)"].replace(0, None)
            # non_zero_heights = df[home_col+"_Height(in cm)"].replace(0, None)
            # non_zero_overall = df[home_col+"_Overall"].replace(0, None)
            # df[home_col+"_Weight"].fillna(non_zero_weights.mean(), inplace=True)
            # df[home_col+"_Height"].fillna(non_zero_heights.mean(), inplace=True)
            
            
            # # Ratings assumption
            # df[home_col+"_Overall"].fillna(non_zero_overall.min(), inplace=True)
            
            # # And for away
            # # Age assumption
            # df[away_col+"_Age"].fillna(18, inplace=True)

            # df[away_col+"_Total"].fillna(18, inplace=True)
            # df[away_col+"_Dribbling Total"].fillna(df[away_col+"_Dribbling Total"].replace(0,None).min(), inplace=True)
            # df[away_col+"_Defending Total"].fillna(df[away_col+"_Defending Total"].replace(0,None).min(), inplace=True)
            # df[away_col+"_Shooting Total"].fillna(df[away_col+"_Shooting Total"].replace(0,None).min(), inplace=True)
            # df[away_col+"_Physicality Total"].fillna(df[away_col+"_Physicality Total"].replace(0,None).min(), inplace=True)
            # df[away_col+"_Pace Total"].fillna(df[away_col+"_Pace Total"].replace(0,None).min(), inplace=True)

            # # Build assumption (ignoring zeros)
            # non_zero_weights_away = df[away_col+"_Weight"].replace(0, None)
            # non_zero_heights_away = df[away_col+"_Height"].replace(0, None)
            # non_zero_overall_away = df[away_col+"_Overall"].replace(0, None)
            # df[away_col+"_Weight"].fillna(non_zero_weights_away.mean(), inplace=True)
            # df[away_col+"_Height"].fillna(non_zero_heights_away.mean(), inplace=True)
            
            # # Ratings assumption
            # df[away_col+"_Overall"].fillna(non_zero_overall_away.min(), inplace=True)
            

    # Dealing with zero values
    for i in range(1, 21):
        home_col = f'HomePlayer{i}'
        away_col = f'AwayPlayer{i}'
        if home_col in df.columns:
            for att in attributes:
                df[home_col+"_"+att].replace(0, None, inplace=True)
        if away_col in df.columns:
            for att in attributes:
                df[away_col+"_"+att].replace(0, None, inplace=True)
            # df[home_col+"_Age"].replace(0, None, inplace=True)
            # df[home_col+"_Weight"].replace(0, None, inplace=True)
            # df[home_col+"_Height"].replace(0, None, inplace=True)
            # df[home_col+"_Overall"].replace(0, None, inplace=True)
            # df[away_col+"_Dribbling Total"].replace(0,None)
            # df[away_col+"_Defending Total"].replace(0,None)
            # df[away_col+"_Shooting Total"].replace(0,None)
            # df[away_col+"_Pace Total"].replace(0,None)
            # df[away_col+"_Passing Total"].replace(0,None)
            # df[away_col+"_Dribbling Total"].replace(0,None)
            # df[away_col+"_Age"].replace(0, None, inplace=True)
            # df[away_col+"_Weight"].replace(0, None, inplace=True)
            # df[away_col+"_Height"].replace(0, None, inplace=True)
            # df[away_col+"_Overall"].replace(0, None, inplace=True)



In [41]:

def set_season_cols(columns, season):
    for c in season.columns:
        for att in ['Overall', 'Age', 'Height(in cm)', 'Weight(in kg)']:
            if att in c:
                columns.append(c)
    



In [42]:
def create_csv(df, columns, name):
    df[columns].to_csv(name)

In [43]:
def pipeline(matches, ratings):
    matches_df = pd.read_csv(matches+'.csv')
    ratings_df=  pd.read_csv(ratings+'.csv')
    ratings_df.rename(columns={'long_name' : 'Full Name', 'height_cm':'Height(in cm)',
                          'weight_kg' : 'Weight(in kg)',
                          'age' : 'Age','club_name' :"Club Name",
                          'overall' : 'Overall', 'dribbling' : 'Dribbling Total',
                          'pace' : 'Pace Total', 'defending' : "Defending Total" ,
                          "shooting" : 'Shooting Total', 'physic' : 'Physicality Total',
                          'passing' : "Passing Total"},inplace=True)
    end_year = int(matches[-2:])
    fix_html_col(matches_df, 'away_fromation')
    fix_html_col(matches_df, 'home_formation')
    
    # missing_values_df = pd.DataFrame(index=ratings_df.index)

    # # Iterate through each column
    # for col in ratings_df.columns:
    #     missing_values_df[col + '_missing'] = ratings_df[col].isnull()

    # Add a new column for the sum of missing values in each row
    #missing_values_df['total_missing'] = missing_values_df.sum(axis=1)

    # Display the DataFrame with missing values information
    #print(missing_values_df['total_missing'])

    names_union = set(show_different_names(matches_df, ratings_df))

    print(names_union)
    
    # Define a dictionary
    names_fix = {'Man Utd': "Manchester United",
             "Man City": 'Manchester City',
             "West Ham": "West Ham United",
             "Nott'm Forest": "Nottingham Forest",
             'Spurs': "Tottenham Hotspur",
             'Wolves': "Wolverhampton Wanderers",
             "Brighton and Hove Albion": "Brighton & Hove Albion",
             "Bournemouth": "AFC Bournemouth",
             'Newcastle': 'Newcastle United',
             'Leicester': 'Leicester City',
             'Leeds': "Leeds United",
             'Huddersfield' : 'Huddersfield Town',
             'Swansea' : 'Swansea City',
             'Cardiff': 'Cardiff City',
             'Norwich' : 'Norwich City',
             'Stoke' : 'Stoke City',
             'West Brom' : 'West Bromwich Albion',
             'Hull' : 'Hull City',
             'QPR' : 'Queens Park Rangers',
             'Sheffield Utd' : 'Sheffield United'}
    

    for name in list(names_union):
        if name not in names_fix.keys():
            print("Fix the name "+name)
            return
    for key, val in names_fix.items():
        matches_df['home_team_name'].replace(key, val, inplace=True)
        matches_df['away_team_name'].replace(key, val, inplace=True)

    # Sanity Check
    

    unique_home_team_names = pd.DataFrame(matches_df['home_team_name'].unique())


    club_names = ratings_df['Club Name'].unique()

    # Check which names in df2 are not in df1
    diff_names = unique_home_team_names[~unique_home_team_names.isin(club_names)]
    #print(diff_names)

    att_list = ['Overall', 'Age', 'Height(in cm)', 'Weight(in kg)']
    for att in att_list:
        if att not in ratings_df.columns:
            print('fix',att)
    

    # Sanity Check
    ratings_df = ratings_df[['Full Name', 'Club Name']+ att_list]
    ratings_col(matches_df, ratings_df)

    replace_nas(matches_df)
    matches_df['Matchweek'] = matches_df['Matchweek'].str.split(' ').str[1].astype(int)

    final_cols = ['home_score', 'home_team_name', 'away_score', 'away_team_name', 'away_fromation', 'home_formation', 'home_GD_prior', 'away_GD_prior', 'home_Points_prior', 'away_Points_prior', 'Matchweek']
    set_season_cols(final_cols, matches_df)
    create_csv(matches_df, final_cols, 'processed Datasets/'+matches+'_proccessed.csv')

In [44]:

pipeline('epl1415', 'players_15')
pipeline('epl1516', 'players_16')
pipeline('epl1617', 'players_17')
pipeline('epl1718', 'players_18')
pipeline('epl1819', 'players_19')
pipeline('epl1920', 'players_20')
pipeline('epl2021', 'players_21')
pipeline('epl2122', 'players_22')
pipeline('epl2223', 'Fifa 23 Players Data')

C:\Users\Yuval\AppData\Local\Temp\ipykernel_102856\2184219997.py:3: DtypeWarning: Columns (104) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_df=  pd.read_csv(ratings+'.csv')


set()
Index(['Full Name', 'Club Name', 'Overall', 'Age', 'Height(in cm)', 'Weight(in kg)'], dtype='object')
(380, 194)


C:\Users\Yuval\AppData\Local\Temp\ipykernel_102856\2184219997.py:3: DtypeWarning: Columns (104) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_df=  pd.read_csv(ratings+'.csv')


{'Bournemouth'}
Index(['Full Name', 'Club Name', 'Overall', 'Age', 'Height(in cm)', 'Weight(in kg)'], dtype='object')
(380, 194)
{'Bournemouth'}
Index(['Full Name', 'Club Name', 'Overall', 'Age', 'Height(in cm)', 'Weight(in kg)'], dtype='object')
(380, 194)


C:\Users\Yuval\AppData\Local\Temp\ipykernel_102856\2184219997.py:3: DtypeWarning: Columns (81,82,83,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_df=  pd.read_csv(ratings+'.csv')


{'Brighton and Hove Albion', 'Bournemouth'}
Index(['Full Name', 'Club Name', 'Overall', 'Age', 'Height(in cm)', 'Weight(in kg)'], dtype='object')
(380, 194)
{'Brighton and Hove Albion', 'Bournemouth'}
Index(['Full Name', 'Club Name', 'Overall', 'Age', 'Height(in cm)', 'Weight(in kg)'], dtype='object')
(380, 194)
{'Brighton and Hove Albion', 'Bournemouth'}
Index(['Full Name', 'Club Name', 'Overall', 'Age', 'Height(in cm)', 'Weight(in kg)'], dtype='object')
(380, 214)
{'Brighton and Hove Albion'}
Index(['Full Name', 'Club Name', 'Overall', 'Age', 'Height(in cm)', 'Weight(in kg)'], dtype='object')
(380, 214)


C:\Users\Yuval\AppData\Local\Temp\ipykernel_102856\2184219997.py:3: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_df=  pd.read_csv(ratings+'.csv')


{'Brighton and Hove Albion'}
Index(['Full Name', 'Club Name', 'Overall', 'Age', 'Height(in cm)', 'Weight(in kg)'], dtype='object')
(380, 214)
{'Brighton and Hove Albion', 'Bournemouth'}
Index(['Full Name', 'Club Name', 'Overall', 'Age', 'Height(in cm)', 'Weight(in kg)'], dtype='object')
(380, 214)
